<a href="https://colab.research.google.com/github/JS-Choi513/Server_IO_prediction_by_machine_learning_method/blob/main/ML_training_note.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import pandas as pd
import numpy as np

In [ ]:
import os
import pandas as pd
import numpy as np
trace_data = pd.read_csv("/content/drive/MyDrive/DATA/CAMRESWMSA03-lvm1.csv")
trace_data = trace_data.drop('Type', axis=1)
trace_data = trace_data.drop('DiskNumber', axis=1)
trace_data = trace_data.drop('Hostname', axis=1)
trace_data = trace_data[['Timestamp', 'Offset','ResponseTime','Size']]
trace_data.boxplot(column=['Size'])
trace_data.hist(column=['Size'])
print(trace_data.head())

Q1 = trace_data.quantile(q=0.25)
Q3 = trace_data.quantile(q=0.75)
print(Q1)
print(Q3)
IQR = Q3-Q1
print(IQR)
IQR_df = trace_data


In [ ]:
'''
IQR_df = trace_data[(trace_data['ResponseTime'] <= Q3['ResponseTime']+1.5*IQR['ResponseTime']) & (trace_data['ResponseTime'] >= Q1['ResponseTime']-1.5*IQR['ResponseTime'])]
IQR_df = IQR_df[(IQR_df['Offset'] <= Q3['Offset']+1.5*IQR['Offset']) & (IQR_df['Offset'] >= Q1['Offset']-1.5*IQR['Offset'])]
IQR_df = IQR_df[(IQR_df['Timestamp'] <= Q3['Timestamp']+1.5*IQR['Timestamp']) & (IQR_df['Timestamp'] >= Q1['Timestamp']-1.5*IQR['Timestamp'])]
IQR_df = IQR_df[(IQR_df['Size'] <= Q3['Size']+1.5*IQR['Size']) & (IQR_df['Size'] >= Q1['Size']-1.5*IQR['Size'])]
IQR_df = IQR_df[['ResponseTime', 'Offset','Timestamp','Size']]

print('original_df :', len(trace_data))
print('IQR_df :', len(IQR_df))
print(IQR_df)
'''

In [ ]:
#IQR_df.boxplot(column=['Size'])


In [ ]:
IQR_df.boxplot(column=['ResponseTime', 'Offset','Timestamp','Size'])
print(IQR_df.head())

In [ ]:
#trace_data.head()
#trace_data.describe()
trace_data.info()

In [ ]:
trace_data_io_encoded = trace_data['Type'].replace(['Read','Write'],[0,1])
trace_data['Type'] = trace_data_io_encoded
print("Min size : ", min(trace_data["Size"]))
print("Max size : ", max(trace_data["Size"]))
print("2^9 ~ 2^20")
#trace_data.plot(kind="line", x = "Timestamp", y="Size")
#trace_data.plot(kind="scatter", x = "Timestamp", y= "Size")
#trace_data.plot(kind="line", x = "Timestamp", y="Offset")
#trace_data.plot(kind="line", x = "Timestamp", y="ResponseTime")
#trace_data.plot(kind="scatter", x = "Size", y="ResponseTime")
#trace_data.plot(kind="scatter", x = "Size", y="ResponseTime", alpha=0.1)
#trace_data.plot(kind="scatter", x = "ResponseTime", y="Offset", alpha=0.1)

In [ ]:
IQR_df['Type'] = trace_data_io_encoded
print("Min size : ", min(IQR_df["Size"]))
print("Max size : ", max(IQR_df["Size"]))
print("2^9 ~ 2^20")

In [ ]:
#trace_data.plot(kind="scatter", x = "ResponseTime", y="Size")
IQR_df.head()

In [ ]:
from pandas.plotting import scatter_matrix
#attributes = ["Timestamp", "Offset",	"Size",	"ResponseTime"]
#scatter_matrix(trace_data[attributes], figsize=(12,8))
from sklearn.model_selection import train_test_split
IQR_df.dropna(axis=0)

train_set, test_set = train_test_split(IQR_df, test_size=0.2)
from sklearn.model_selection import StratifiedShuffleSplit
#split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
#for train_index, test_index in split.split(train_set, train_set["Size"]):
#  strat_train_set = train_set.loc[train_index]
#  strat_test_set = train_set.loc[test_index]
#trace = train_set.drop("DiskNumber", axis=1)
#trace = trace.drop("Hostname", axis=1)
#trace = trace.drop("Type", axis=1)
train_set = train_set[:300000]
print(train_set.head())
trace_label = train_set["Size"].copy()
trace_label = trace_label[:300000]
#test_trace = test_set.drop("DiskNumber", axis=1)
#test_trace = test_trace.drop("Hostname", axis=1)
#test_trace = test_trace.drop("Type", axis=1)
test_trace = test_set[:1000]
test_label = test_set["Size"].copy()
test_label = test_label[:1000]
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
#lin_reg = LinearRegression()
#lin_reg.fit(trace, trace_label)
#request_size_prediction = lin_reg.predict(trace)
#lin_mse = mean_squared_error(trace_label, request_size_prediction)
#lin_rmse = np.sqrt(lin_mse)
#lin_rmse

In [ ]:
from sklearn.preprocessing import OneHotEncoder

#trace_data_io_encoded = encoder.fit_transform(trace_data_io)

train_set.head()

In [ ]:

from sklearn.preprocessing import MinMaxScaler
# Input scale


scaler_x = MinMaxScaler()
scaler_x.fit(train_set.iloc[:, :-1])

train_set.iloc[:, :-1] = scaler_x.transform(train_set.iloc[:, :-1])
test_set.iloc[:, :-1] = scaler_x.transform(test_set.iloc[:, :-1])

# Output scale
scaler_y = MinMaxScaler()
#scaler_y.fit(trace.iloc[:, [-1]])
scaler_y.fit(train_set.iloc[:, [-1]])

train_set.iloc[:, -1] = scaler_y.transform(train_set.iloc[:, [-1]])
test_set.iloc[:,-1] = scaler_y.transform(test_set.iloc[:, [-1]])

print(train_set.head())
#print(test_trace.head)

train_set.boxplot(column=['ResponseTime', 'Offset','Timestamp','Size'])
#test_trace.boxplot(column=['ResponseTime','Offset','Timestamp','Size'])

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
device = torch.device("cpu")

torch.manual_seed(1)

In [ ]:
from torch.utils.data import TensorDataset 
from torch.utils.data import DataLoader 

# 데이터셋 생성 
def build_dataset(time_series, seq_length):
    dataX = []
    dataY = []
    for i in range(0, len(time_series)-seq_length):
        _x = time_series[i:i+seq_length, :]
        _y = time_series[i+seq_length, [2]]
        #print(_x, "-->",_y)
        dataX.append(_x)
        dataY.append(_y)

    return np.array(dataX), np.array(dataY)

trainX, trainY = build_dataset(np.array(train_set), 10)
testX, testY = build_dataset(np.array(test_set), 10)
print(trainX.shape)
print(trainY.shape)
trainX_tensor = torch.FloatTensor(trainX).to(device)
trainY_tensor = torch.FloatTensor(trainY).to(device)

print(trainX_tensor)
print(trainY_tensor)

testX_tensor = torch.FloatTensor(testX).to(device)
testY_tensor = torch.FloatTensor(testY).to(device)

dataset = TensorDataset(trainX_tensor, trainY_tensor)

dataloader = DataLoader(dataset,
                        batch_size=256,
                        shuffle=True,  
                        drop_last=True)

In [ ]:
data_dim = 4
hidden_dim = 8
output_dim = 1 
learning_rate = 0.0001
nb_epochs = 100
device = torch.device("cuda")

class Net(nn.Module):
    def __init__(self, input_dim, hidden_dim, seq_len, output_dim, layers):
        super(Net, self).__init__()
        self.hidden_dim = hidden_dim
        self.seq_len = seq_len
        self.output_dim = output_dim
        self.layers = layers
        
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers=layers,
                            # dropout = 0.1,
                            batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim, bias = True) 
        
    def reset_hidden_state(self): 
        self.hidden = (
                torch.zeros(self.layers, self.seq_len, self.hidden_dim),
                torch.zeros(self.layers, self.seq_len, self.hidden_dim))
        
    def forward(self, x):
        x, _status = self.lstm(x)
        x = self.fc(x[:, -1])
        return x

In [ ]:

def train_model(model, train_df, num_epochs = None, lr = None, verbose = 10, patience = 10):
     
    criterion = nn.MSELoss().to(device)
    optimizer = optim.Adam(model.parameters(), lr = 0.0001)
    nb_epochs = num_epochs
    
    train_hist = np.zeros(nb_epochs)

    for epoch in range(nb_epochs):
        avg_cost = 0
        total_batch = len(train_df)
        
        for batch_idx, samples in enumerate(train_df):

            x_train, y_train = samples
            
            model.reset_hidden_state()
            
            outputs = model(x_train)
                
            loss = torch.sqrt(criterion(outputs, y_train))
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            avg_cost += loss/total_batch
               
        train_hist[epoch] = avg_cost        
        
        if epoch % verbose == 0:
            print('Epoch:', '%04d' % (epoch), 'train loss :', '{:.4f}'.format(avg_cost))
        if (epoch % patience == 0) & (epoch != 0):
            if train_hist[epoch-patience] < train_hist[epoch]:
                print('\n Early Stopping')
               
                break
            
    return model.eval(), train_hist

In [ ]:


net = Net(data_dim, hidden_dim, 10, output_dim, 4)#.to(device)
model, train_hist = train_model(net, dataloader, num_epochs = nb_epochs, lr = learning_rate, verbose = 20, patience = 10)

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure(figsize=(10, 4))
plt.plot(train_hist, label="Training loss")
plt.legend()
plt.show()

In [ ]:

with torch.no_grad(): 
    pred = []
    for pr in range(len(testX_tensor)):

        model.reset_hidden_state()

        predicted = model(torch.unsqueeze(testX_tensor[pr], 0))
        predicted = torch.flatten(predicted).item()
        pred.append(predicted)
    
    device = torch.device("cpu")
  
    pred_inverse = np.array(pred).reshape(-1, 1)
    testY_inverse = np.array(testY_tensor)


In [ ]:
fig = plt.figure(figsize=(8,3))
plt.plot(np.arange(len(pred_inverse)), pred_inverse, label = 'pred')
plt.title("Prediction plot")
plt.show()

In [ ]:
fig = plt.figure(figsize=(8,3))
plt.plot(np.arange(len(testY_inverse)), testY_inverse, label = 'true')
plt.title("Label plot")
plt.show()

In [ ]:
trace_data.plot(kind="scatter", x = "Type", y="ResponseTime", alpha=0.1)
trace_data.plot(kind="scatter", x ="Type", y="Size", alpha=0.1,
                c="Offset",  label="Request size", 
                 cmap=plt.get_cmap("jet"), colorbar=True)
plt.legend()
trace_data.plot(kind="scatter", x = "ResponseTime", y="Type")

In [ ]:
from sklearn.model_selection import train_test_split
trace_data.dropna(axis=0)

train_set, test_set = train_test_split(trace_data, test_size=0.2, random_state=42)
from sklearn.model_selection import StratifiedShuffleSplit
trace = train_set.drop("Size", axis=1)
trace = trace.drop("Hostname", axis=1)
trace_label = train_set["Size"].copy()
test_trace = test_set.drop("Size", axis=1)
test_trace = test_trace.drop("Hostname", axis=1)
test_label = test_set["Size"].copy()
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
lin_reg = LinearRegression()
lin_reg.fit(trace, trace_label)
request_size_prediction = lin_reg.predict(trace)
lin_mse = mean_squared_error(trace_label, request_size_prediction)
lin_rmse = np.sqrt(lin_mse)
lin_rmse

In [ ]:
from sklearn.tree import DecisionTreeRegressor
tree_reg = DecisionTreeRegressor()
tree_reg.fit(trace, trace_label)
size_predictions = tree_reg.predict(trace)
tree_mse = mean_squared_error(trace_label, size_predictions)
tree_rmse = np.sqrt(tree_mse)
print(tree_rmse)

In [ ]:
from sklearn.model_selection import cross_val_score
def display_score(scores):
  print("Scores:", scores)
  print("Mean:", scores.mean())
  print("Standard deviation:", scores.std())

scores = cross_val_score(tree_reg, trace, trace_label, scoring="neg_mean_squared_error", cv=10)
rmse_scores = np.sqrt(-scores)
display_score(rmse_scores)
print("")
lin_scores = cross_val_score(lin_reg, trace, trace_label, scoring="neg_mean_squared_error", cv=10)
lin_rmse_scores = np.sqrt(-lin_scores)
display_score(lin_rmse_scores)

In [ ]:
import seaborn as sns
from sklearn.metrics import mean_squared_error

my_predictions = {}
my_pred = None
my_actual = None
my_name = None

colors = ['r', 'c', 'm', 'y', 'k', 'khaki', 'teal', 'orchid', 'sandybrown',
          'greenyellow', 'dodgerblue', 'deepskyblue', 'rosybrown', 'firebrick',
          'deeppink', 'crimson', 'salmon', 'darkred', 'olivedrab', 'olive', 
          'forestgreen', 'royalblue', 'indigo', 'navy', 'mediumpurple', 'chocolate',
          'gold', 'darkorange', 'seagreen', 'turquoise', 'steelblue', 'slategray', 
          'peru', 'midnightblue', 'slateblue', 'dimgray', 'cadetblue', 'tomato'
         ]

def plot_predictions(name_, pred, actual):
    df = pd.DataFrame({'prediction': pred, 'actual': test_label})
    df = df.sort_values(by='actual').reset_index(drop=True)

    plt.figure(figsize=(11, 8))
    plt.scatter(df.index, df['prediction'], marker='x', color='r')
    
    plt.scatter(df.index, df['actual'], alpha=0.7, marker='o', color='black')
    plt.ylabel('Request Size')
    plt.xlabel('Index')
    plt.title(name_, fontsize=15)
    plt.legend(['prediction', 'actual'], fontsize=12)
    plt.show()

def rmse_eval(name_, pred, actual):
    global my_predictions, colors, my_pred, my_actual, my_name
    
    my_name = name_
    my_pred = pred
    my_actual = actual

    plot_predictions(name_, pred, actual)

    mse =np.sqrt( mean_squared_error(pred, actual))
    my_predictions[name_] = mse

    y_value = sorted(my_predictions.items(), key=lambda x: x[1], reverse=True)
    
    df = pd.DataFrame(y_value, columns=['model', 'rmse'])
    print(df)
    min_ = df['rmse'].min() - 10
    max_ = df['rmse'].max() + 10
    
    length = len(df) / 2

    plt.figure(figsize=(9, length))
    ax = plt.subplot()
    ax.set_yticks(np.arange(len(df)))
    ax.set_yticklabels(df['model'], fontsize=12)
    bars = ax.barh(np.arange(len(df)), df['rmse'], height=0.3)
    
    for i, v in enumerate(df['rmse']):
        idx = np.random.choice(len(colors))
        bars[i].set_color(colors[idx])
        ax.text(v + 2, i, str(round(v, 3)), color='k', fontsize=12, fontweight='bold', verticalalignment='center')
        
    plt.title('RMSE Error', fontsize=16)
    plt.xlim(min_, max_)
    
    plt.show()
    
def add_model(name_, pred, actual):
    global my_predictions, my_pred, my_actual, my_name
    my_name = name_
    my_pred = pred
    my_actual = actual
    
    mse = mean_squared_error(pred, actual)
    my_predictions[name_] = mse

def remove_model(name_):
    global my_predictions
    try:
        del my_predictions[name_]
    except KeyError:
        return False
    return True

def plot_all():
    global my_predictions, my_pred, my_actual, my_name
    
    plot_predictions(my_name, my_pred, my_actual)
    
    y_value = sorted(my_predictions.items(), key=lambda x: x[1], reverse=True)
    
    df = pd.DataFrame(y_value, columns=['model', 'rmse'])
    print(df)
    min_ = df['rmse'].min() - 10
    max_ = df['rmse'].max() + 10
    
    length = len(df) / 2
    
    plt.figure(figsize=(9, length))
    ax = plt.subplot()
    ax.set_yticks(np.arange(len(df)))
    ax.set_yticklabels(df['model'], fontsize=12)
    bars = ax.barh(np.arange(len(df)), df['rmse'], height=0.3)
    
    for i, v in enumerate(df['rmse']):
        idx = np.random.choice(len(colors))
        bars[i].set_color(colors[idx])
        ax.text(v + 2, i, str(round(v, 3)), color='k', fontsize=12, fontweight='bold', verticalalignment='center')
        
    plt.title('RMSE Error', fontsize=16)
    plt.xlim(min_, max_)
    
    plt.show()

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet

In [ ]:
linear_reg = LinearRegression()
linear_reg.fit(trace, trace_label)
pred = linear_reg.predict(test_trace)
rmse_eval('LinearRegression', pred, test_label)

In [ ]:
ridge = Ridge(alpha=0.1)
ridge.fit(trace, trace_label)
pred = ridge.predict(test_trace)
rmse_eval('Ridge(alpha=0.1)', pred, test_label)

In [ ]:
lasso = Lasso(alpha=0.01)
lasso.fit(trace, trace_label)
pred = lasso.predict(test_trace)
rmse_eval('Lasso(alpha=0.01)', pred, test_label)

In [ ]:
elasticnet = ElasticNet(alpha=0.01, l1_ratio=0.8)
elasticnet.fit(trace, trace_label)
pred = elasticnet.predict(test_trace)
rmse_eval('ElasticNet(alpha=0.1, l1_ratio=0.8)', pred, test_label)   

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures
elasticnet_pipeline = make_pipeline(
    StandardScaler(),
    ElasticNet(alpha=0.01, l1_ratio=0.8)
)
elasticnet_pipeline.fit(trace, trace_label)
elasticnet_pred = elasticnet_pipeline.predict(test_trace)
rmse_eval('Standard ElasticNet', elasticnet_pred, test_label)

In [ ]:
poly_pipeline = make_pipeline(
    PolynomialFeatures(degree=2, include_bias=False),
    ElasticNet(alpha=0.1, l1_ratio=0.2)
)
poly_pipeline.fit(trace, trace_label)
poly_pred = poly_pipeline.predict(test_trace)
rmse_eval('Poly ElasticNet', poly_pred, test_label)

In [ ]:
from sklearn.ensemble import VotingRegressor, VotingClassifier
single_models = [
    ('linear_reg', linear_reg), 
    ('ridge', ridge), 
    ('lasso', lasso), 
    ('elasticnet_pipeline', elasticnet_pipeline), 
    ('poly_pipeline', poly_pipeline)
]
voting_regressor = VotingRegressor(single_models, n_jobs=-1)
voting_regressor.fit(trace, trace_label)


In [ ]:
voting_pred = voting_regressor.predict(test_trace)
rmse_eval('Voting Ensemble', voting_pred, test_label)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rfr = RandomForestRegressor(random_state=42, n_estimators=1000, max_depth=7, max_features=0.9)

rfr.fit(trace, trace_label)
rfr_pred = rfr.predict(test_trace)


In [ ]:
rmse_eval('RandomForest Ensemble', rfr_pred, test_label)

In [ ]:
from sklearn.svm import SVC, SVR
svr = SVR(kernel='rbf', gamma='auto')
svr.fit(trace, trace_label)
score = svr.score(trace, trace_label)
print("R-squared: ", score)

cv_score = cross_val_score(svr, trace, trace_label, cv=5)
print("CV mean score: ", cv_score.mean())

ypred = svr.predict(test_trace)

mse = mean_squared_error(test_label, ypred)
print("MSE: ", mse)

x_ax = range(len(test_label))
plt.plot(x_ax, test_trace, label='origianl')
plt.plot(x_ax, ypred, label="predicted")
plt.title("Botson test and predicted data")
plt.legend()
plt.show()